In [1]:
#import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



### deribit API Request

In [2]:
""" Download CC-Option Data from Deribit via public API """

"""
Matteo Bottacini -- matteo.bottacini@usi.ch
"""

# import modules
import json
import requests
from tqdm import tqdm
import sqlite3
import datetime


# functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """

    # requests public API
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin
    """

    # get option name and settlement
    coin_name = get_option_name_and_settlement(coin)[0]
    settlement_period = get_option_name_and_settlement(coin)[1]

    # initialize data frame
    coin_df = []

    # initialize progress bar
    pbar = tqdm(total=len(coin_name))

    # loop to download data for each Option Name
    for i in range(len(coin_name)):
        # download option data -- requests and convert json to pandas
        r = requests.get('https://test.deribit.com/api/v2/public/get_order_book?instrument_name=' + coin_name[i])
        result = json.loads(r.text)
        df = pd.json_normalize(result['result'])

        # add settlement period
        df['settlement_period'] = settlement_period[i]

        # append data to data frame
        coin_df.append(df)

        # update progress bar
        pbar.update(1)

    # finalize data frame
    coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df




In [3]:
# print data and time for log
print('Date and time: ' +  datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' , format: dd/mm/yyyy hh:mm:ss')

# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')
eth_data = get_option_data('ETH')

# The data is now stored in Pandas DataFrames 'btc_data' and 'eth_data'
print('BTC data collected')
print(btc_data.head()) # Display first few rows of BTC data
print('ETH data collected')
print(eth_data.head()) # Display first few rows of ETH data

Date and time: 02/05/2024 21:43:40 , format: dd/mm/yyyy hh:mm:ss


100%|██████████| 1030/1030 [06:32<00:00,  2.80it/s]C:\Users\ethan\AppData\Local\Temp\ipykernel_25272\1058301515.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
100%|██████████| 948/948 [06:14<00:00,  2.77it/s]C:\Users\ethan\AppData\Local\Temp\ipykernel_25272\1058301515.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
100%|██████████| 948/948 [06:14<00:00,  2.53it/s]

BTC data collected
   best_bid_amount  best_ask_amount  bid_iv  ask_iv underlying_index  \
0              0.0              0.0     0.0    0.00       BTC-3MAY24   
0              0.0             10.0     0.0    0.00       BTC-3MAY24   
0              0.0              0.0     0.0    0.00       BTC-3MAY24   
0              0.0             13.4     0.0  513.66       BTC-3MAY24   
0              0.0             10.0     0.0  465.65       BTC-3MAY24   

   underlying_price  mark_iv  best_bid_price  best_ask_price  interest_rate  \
0          59330.12    100.0             0.0          0.0000            0.0   
0          59330.12    100.0             0.0          0.0001            0.0   
0          59329.29    100.0             0.0          0.0000            0.0   
0          59329.29    100.0             0.0          0.0001            0.0   
0          59327.17    100.0             0.0          0.2425            0.0   

   ...  greeks.theta  greeks.vega  greeks.gamma  greeks.delta  \
0  ...  

In [4]:
btc_data.to_csv('data/btc_data.csv', index=False)
eth_data.to_csv('data/eth_data.csv', index=False)

In [5]:
btc_data.head()

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,best_bid_price,best_ask_price,interest_rate,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,0.0,0.0,0.00,BTC-3MAY24,59330.12,100.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,1.0,402682.41,18.0,0.0000,0.3850,0.3850,week
0,0.0,10.0,0.0,0.00,BTC-3MAY24,59330.12,100.0,0.0,0.0001,0.0,...,0.0,0.0,0.0,0.0,4.74,0.8,0.0000,0.0001,0.0001,week
0,0.0,0.0,0.0,0.00,BTC-3MAY24,59329.29,100.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,1.0,277903.56,16.0,-6.6667,0.2800,0.3000,week
0,0.0,13.4,0.0,513.66,BTC-3MAY24,59329.29,100.0,0.0,0.0001,0.0,...,0.0,0.0,0.0,0.0,40.17,6.8,0.0000,0.0001,0.0001,week
0,0.0,10.0,0.0,465.65,BTC-3MAY24,59327.17,100.0,0.0,0.2425,0.0,...,0.0,0.0,0.0,1.0,209329.77,24.0,146.9136,0.0810,0.2000,week


In [6]:
btc_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1030 entries, 0 to 0
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   best_bid_amount     1030 non-null   float64
 1   best_ask_amount     1030 non-null   float64
 2   bid_iv              1030 non-null   float64
 3   ask_iv              1030 non-null   float64
 4   underlying_index    1030 non-null   object 
 5   underlying_price    1030 non-null   float64
 6   mark_iv             1030 non-null   float64
 7   best_bid_price      1030 non-null   float64
 8   best_ask_price      1030 non-null   float64
 9   interest_rate       1030 non-null   float64
 10  open_interest       1030 non-null   float64
 11  max_price           1030 non-null   float64
 12  min_price           1030 non-null   float64
 13  last_price          865 non-null    float64
 14  asks                1030 non-null   object 
 15  bids                1030 non-null   object 
 16  settlement_pri

In [7]:
eth_data.head()

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,best_bid_price,best_ask_price,interest_rate,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,1050.0,0.00,561.89,ETH-3MAY24,2946.23,151.81,0.0,0.2540,0.0,...,0.00000,0.00000,0.00000,1.00000,0.00,0.0,NaN,NaN,NaN,week
0,0.0,2.0,0.00,379.84,ETH-3MAY24,2946.16,151.81,0.0,0.0002,0.0,...,0.00000,0.00000,0.00000,0.00000,54.32,186.0,0.0000,0.0001,0.0001,week
0,0.0,0.0,939.08,0.00,ETH-3MAY24,2946.23,134.52,0.0,0.0000,0.0,...,0.00000,0.00000,0.00000,1.00000,345278.77,500.0,14.4914,0.2153,0.2465,week
0,0.0,0.0,164.92,222.51,ETH-3MAY24,2946.16,134.52,0.0,0.0000,0.0,...,-0.58565,0.03727,0.00006,-0.00597,0.00,0.0,NaN,NaN,NaN,week
0,0.0,1050.0,0.00,480.24,ETH-3MAY24,2946.19,102.41,0.0,0.1875,0.0,...,0.00000,0.00000,0.00000,1.00000,0.00,0.0,NaN,NaN,NaN,week


In [8]:
eth_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 0
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   best_bid_amount     948 non-null    float64
 1   best_ask_amount     948 non-null    float64
 2   bid_iv              948 non-null    float64
 3   ask_iv              948 non-null    float64
 4   underlying_index    948 non-null    object 
 5   underlying_price    948 non-null    float64
 6   mark_iv             948 non-null    float64
 7   best_bid_price      948 non-null    float64
 8   best_ask_price      948 non-null    float64
 9   interest_rate       948 non-null    float64
 10  open_interest       948 non-null    float64
 11  max_price           948 non-null    float64
 12  min_price           948 non-null    float64
 13  last_price          784 non-null    float64
 14  asks                948 non-null    object 
 15  bids                948 non-null    object 
 16  settlement_pric